In [1]:
from pseudoproof.ml_logic.model import gbc_model
# from pseudoproof.ml_logic.preproc import *
from pseudoproof.ml_logic.preproc import clean_data,scale_data,digit_freq
import pandas as pd
import joblib

from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

contents = pd.read_csv("/Users/anaispepey/code/APepey/PseudoProof/raw_data/datasets/complete_dataset_true_fake.csv")


2023-12-01 17:23:00.951163: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## working workflow:
### to do: retrain models on final training dataset
### add same structure for RNN


# First 5 Models

In [2]:
#parameters to be updated when gridsearch is complete
knn_model = KNeighborsClassifier(n_neighbors=5)
nb_model = GaussianNB()
gbc_model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1)
svm_model = SVC(kernel="rbf", C=1, random_state=42)
rf_model = RandomForestClassifier(n_estimators=100)

MODELS = {
    knn_model: "knn",
    nb_model: "naive_bayes",
    gbc_model: "gradient_boosting",
    rf_model: "random_forest",
    svm_model: "svm",
}


In [3]:
df = clean_data(contents)

X = df.drop(columns='y')
y = df["y"]

X_scaled = scale_data(X)
X_final = digit_freq(X_scaled)

# split

# X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_split=0.3)

X_train, X_test = X_final[:-1], X_final[-1:]
y_train, y_test = y[:-1], y[-1:]


✅ data cleaned of NAs and duplicates
✅ data scaled
✅ digits frequency computed 
final shape: (374, 20)


In [4]:
X_train.sample()


,f_0,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,s_0,s_1,s_2,s_3,s_4,s_5,s_6,s_7,s_8,s_9
254,0.0,0.0,0.05,0.1,0.0,0.05,0.0,0.1,0.0,0.0,0.1,0.0,0.05,0.0,0.0,0.05,0.0,0.1,0.0,0.0


In [5]:
knn_model = knn_model.fit(X_train, y_train)
nb_model = nb_model.fit(X_train, y_train)
gbc_model = gbc_model.fit(X_train, y_train)
svm_model = svm_model.fit(X_train, y_train)
rf_model = rf_model.fit(X_train, y_train)


In [6]:
key_value_pairs = MODELS.items()

for key, value in key_value_pairs:
    model_path = f"/Users/anaispepey/code/APepey/PseudoProof/trained_models/{value}.pkl"
    joblib.dump(key, filename=model_path)


#last step: drag all <model>.pkl files to pseudoproof Google Bucket


# RNN model - to do

#train RNN model
#save it in trained_models
#drag it to google bucket 'pseudoproof'
#load it into local machine